#Data Engineering with Pandas: Part 1
Best Practices with Matt Harrison
https://www.youtube.com/watch?v=zgbUk90aQ6A 

###Data
Fuel economy data are the result of vehicle testing done at the Environmental Protection Agency's National Vehicle and Fuel Emissions Laboratory in Ann Arbor, Michigan, and by vehicle manufacturers with oversight by EPA.
https://www.fueleconomy.gov/feg/download.shtml 

###Problems
* Data is stored in .csv format, but .csv does not record data type, so pandas must infer.
* Bad data typing can increase storage, memory, and processing costs.
* Incorrect data typing can disable machine learning algorithms. 

###Objectives
* Select columns of interest from dataset.
* Conduct Exploratory Data Analysis, including data type.
* Optimize storage, memory, and processing costs with data type improvements.
* Use flow programming to transform data for explainability and repeatability.

###Approach
In general, a best practice in data engineering is to work on the original dataset, making no destructive changes, and chaining operations, imitating a recipe, for explainability. 

After testing and verification, the data pipelines will be expressed in a function for repeatability. 

###**Outcome**

A repeatable function was developed that applies the correct data type to columns of interest, re-engineers other problematic columns, and reduces a 19.6MB dataset to 8.7MB, achieving a **memory savings of 56 percent**.


##Load Data


In [ ]:
%matplotlib inline
from IPython.display import display
import numpy as np
import pandas as pd

In [ ]:
pd.__version__

'1.3.5'

In [ ]:
pd.options.display.min_rows = 20

In [ ]:
autos = pd.read_csv('https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
autos.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,...,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,2000,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,...,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-2250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3850,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,...,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-11500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1550,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,...,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3850,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,...,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-11500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2700,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,...,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-5750,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [ ]:
autos.shape

(41144, 83)

##Data Types
Getting the right type enables analysis and ensures correctness.

In this section:
* What data type is each column?
* How much memory does each consume?
* How much total memory do all consume? 

In [ ]:
cols = autos.columns

In [ ]:
cols = cols.to_list()

In [ ]:
cols[0:5]

['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08']

In [ ]:
cols_subset = [
 'city08',
 'comb08',
 'highway08',
 'cylinders',
 'displ',
 'drive',
 'eng_dscr',
 'fuelCost08',
 'make',
 'model',
 'range',
 'trany',
 'createdOn',
 'year']

In [ ]:
autos[cols_subset].dtypes
#pandas makes python fast by leveraging numpy. 
#Numpy makes math in python possible with C speeds.
#Examples: 
# - int64 is a numpy type not available in python.
# - int64 only applied if no missing values, else float64. 
#    - First clue there may be missing values.
#    - Machine learning may choke if missing values.
#    - Machine learning is speedier with int64 where possible.
# - object points to python objects instead of using buffers for memory (slowest type)
#    - used for mixed type, string, categorical, pre-formed text, or boolean.   

city08          int64
comb08          int64
highway08       int64
cylinders     float64
displ         float64
drive          object
eng_dscr       object
fuelCost08      int64
make           object
model          object
range           int64
trany          object
createdOn      object
year            int64
dtype: object

In [ ]:
autos[cols_subset].memory_usage(deep=True)
#Deep equals true produces a reading of objects. 

Index             128
city08         329152
comb08         329152
highway08      329152
cylinders      329152
displ          329152
drive         3028369
eng_dscr      2135693
fuelCost08     329152
make          2606267
model         2813134
range          329152
trany         2933276
createdOn     3497240
year           329152
dtype: int64

In [ ]:
autos[cols_subset].memory_usage(deep=True).sum()
#19 million, 647 thousand, 323. Equals 19+ MegaBytes(MB)

19647323

## Ints
Pandas and numpy have integer types that python does not. Int8 and Int16 both improve on python by adding near C speed to machine learning with python.

To determine the right data type: 
* Compare the data type to the statistical range. Does one extent match the other?
* Change type if appropriate for optimum memory consumption.

In [ ]:
#Statistically describe only columns with integers.
#autos[cols_subset].select_dtypes(int).describe()
#Can be written as self-executing expression.
descriptive_stats = (autos
                      [cols_subset]
                      .select_dtypes(int)
                      .describe()
                    )
descriptive_stats

,city08,comb08,highway08,fuelCost08,range,year
count,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000
mean,18.369045,20.616396,24.504667,2362.335942,0.793506,2001.535266
std,7.905886,7.674535,7.730364,654.981925,13.041592,11.142414
min,6.000000,7.000000,9.000000,500.000000,0.000000,1984.000000
25%,15.000000,17.000000,20.000000,1900.000000,0.000000,1991.000000
50%,17.000000,20.000000,24.000000,2350.000000,0.000000,2002.000000
75%,20.000000,23.000000,28.000000,2700.000000,0.000000,2011.000000
max,150.000000,136.000000,124.000000,7400.000000,370.000000,2020.000000


* count is the nbr of non-missing values.
* min to max is the range.
* 50% is the median value.
* columns can be indexed.

In [ ]:
#Could a smaller type be used without losing precision? Ex: highway08? 
#Answer: Yes. Notice that abs(min) and max > abs(actual values).
np.iinfo(np.int8)

iinfo(min=-128, max=127, dtype=int8)

In [ ]:
np.iinfo(np.int16)

iinfo(min=-32768, max=32767, dtype=int16)

In [ ]:
#Pass object with key/value pairs to retype columns.
#Pass list to select_type for multiple selections.
(autos
 [cols_subset]
 .astype({'highway08':'int8','city08':'int16','comb08':'int16'})
 .select_dtypes([int,'int8'])
 .describe()
)

,highway08,fuelCost08,range,year
count,41144.000000,41144.000000,41144.000000,41144.000000
mean,24.504667,2362.335942,0.793506,2001.535266
std,7.730364,654.981925,13.041592,11.142414
min,9.000000,500.000000,0.000000,1984.000000
25%,20.000000,1900.000000,0.000000,1991.000000
50%,24.000000,2350.000000,0.000000,2002.000000
75%,28.000000,2700.000000,0.000000,2011.000000
max,124.000000,7400.000000,370.000000,2020.000000


In [ ]:
#Re-calculate memory usage and compare to pre-.
(autos[cols_subset]
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16'})
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

18124995

## Floats
* Can any floats be converted to integers?
* Were any floats assigned due to missing values?

In [ ]:
(autos
 [cols_subset]
 .select_dtypes('float')
 ).head()

,cylinders,displ
0,4.0,2.0
1,12.0,4.9
2,4.0,2.2
3,8.0,5.2
4,4.0,2.2


It looks as if cylinders is mis-typed. 
Are there missing values?

In [ ]:
print(autos.cylinders.describe())
print(autos.cylinders.shape[0], autos.cylinders.count())

count    40938.000000
mean         5.717084
std          1.755517
min          2.000000
25%          4.000000
50%          6.000000
75%          6.000000
max         16.000000
Name: cylinders, dtype: float64
41144 40938


In [ ]:
#.value_counts also reveals missing values
autos.cylinders.value_counts(dropna=False)
#Notice that 206 are NaN and all others integers.

4.0     15938
6.0     14284
8.0      8801
5.0       771
12.0      626
3.0       279
NaN       206
10.0      170
2.0        59
16.0       10
Name: cylinders, dtype: int64

In [ ]:
missing_cylinders = autos.cylinders.isna()
autos_abbr = autos[cols_subset]

In [ ]:
#Do the missing values occur because electric cars have no cylinders?
autos_abbr[autos_abbr.cylinders.isna()].head(10)
#ANSWER: Yes, with very few exceptions.

,city08,comb08,highway08,cylinders,displ,drive,eng_dscr,fuelCost08,make,model,range,trany,createdOn,year
7138,81,85,91,NaN,NaN,NaN,NaN,800,Nissan,Altra EV,90,NaN,Tue Jan 01 00:00:00 EST 2013,2000
7139,81,72,64,NaN,NaN,2-Wheel Drive,NaN,900,Toyota,RAV4 EV,88,NaN,Tue Jan 01 00:00:00 EST 2013,2000
8143,81,72,64,NaN,NaN,2-Wheel Drive,NaN,900,Toyota,RAV4 EV,88,NaN,Tue Jan 01 00:00:00 EST 2013,2001
8144,74,65,58,NaN,NaN,NaN,NaN,1000,Ford,Th!nk,29,NaN,Tue Jan 01 00:00:00 EST 2013,2001
8146,45,39,33,NaN,NaN,2-Wheel Drive,NaN,1700,Ford,Explorer USPS Electric,38,NaN,Tue Jan 01 00:00:00 EST 2013,2001
8147,84,75,66,NaN,NaN,NaN,NaN,900,Nissan,Hyper-Mini,33,NaN,Tue Jan 01 00:00:00 EST 2013,2001
9212,87,78,69,NaN,NaN,2-Wheel Drive,NaN,850,Toyota,RAV4 EV,95,NaN,Tue Jan 01 00:00:00 EST 2013,2002
9213,45,39,33,NaN,NaN,2-Wheel Drive,NaN,1700,Ford,Explorer USPS Electric,38,NaN,Tue Jan 01 00:00:00 EST 2013,2002
10329,87,78,69,NaN,NaN,2-Wheel Drive,NaN,850,Toyota,RAV4 EV,95,NaN,Tue Jan 01 00:00:00 EST 2013,2003
21413,22,24,28,NaN,NaN,4-Wheel Drive,NaN,1750,Subaru,RX Turbo,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1985


## Replacing Missing Values
Having enough information now, we can replace missing values. 

Create mini pipelines in pandas by passing column names and new values to assign and by chaining other methods.

In [ ]:
#Electric vehicles should have none (0). 
#Pass fillna(0) into assign and retype the column as int8.
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0))
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16'})
 .describe()
 )

,city08,comb08,highway08,cylinders,displ,fuelCost08,range,year
count,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000,41144.000000
mean,18.369045,20.616396,24.504667,5.688460,3.277904,2362.335942,0.793506,2001.535266
std,7.905886,7.674535,7.730364,1.797009,1.373415,654.981925,13.041592,11.142414
min,6.000000,7.000000,9.000000,0.000000,0.000000,500.000000,0.000000,1984.000000
25%,15.000000,17.000000,20.000000,4.000000,2.200000,1900.000000,0.000000,1991.000000
50%,17.000000,20.000000,24.000000,6.000000,3.000000,2350.000000,0.000000,2002.000000
75%,20.000000,23.000000,28.000000,6.000000,4.300000,2700.000000,0.000000,2011.000000
max,150.000000,136.000000,124.000000,16.000000,8.400000,7400.000000,370.000000,2020.000000


In [ ]:
autos[cols_subset].describe()

,city08,comb08,highway08,cylinders,displ,fuelCost08,range,year
count,41144.000000,41144.000000,41144.000000,40938.000000,40940.000000,41144.000000,41144.000000,41144.000000
mean,18.369045,20.616396,24.504667,5.717084,3.294238,2362.335942,0.793506,2001.535266
std,7.905886,7.674535,7.730364,1.755517,1.357151,654.981925,13.041592,11.142414
min,6.000000,7.000000,9.000000,2.000000,0.000000,500.000000,0.000000,1984.000000
25%,15.000000,17.000000,20.000000,4.000000,2.200000,1900.000000,0.000000,1991.000000
50%,17.000000,20.000000,24.000000,6.000000,3.000000,2350.000000,0.000000,2002.000000
75%,20.000000,23.000000,28.000000,6.000000,4.300000,2700.000000,0.000000,2011.000000
max,150.000000,136.000000,124.000000,16.000000,8.400000,7400.000000,370.000000,2020.000000


In [ ]:
#Use iinfo for integers and finfo for floats.
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [ ]:
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'))
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16'})
 ).head(10)

,city08,comb08,highway08,cylinders,displ,drive,eng_dscr,fuelCost08,make,model,range,trany,createdOn,year
0,19,21,25,4,2.000000,Rear-Wheel Drive,(FFS),2000,Alfa Romeo,Spider Veloce 2000,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1985
1,9,11,14,12,4.898438,Rear-Wheel Drive,(GUZZLER),3850,Ferrari,Testarossa,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1985
2,23,27,33,4,2.199219,Front-Wheel Drive,(FFS),1550,Dodge,Charger,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1985
3,10,11,12,8,5.199219,Rear-Wheel Drive,NaN,3850,Dodge,B150/B250 Wagon 2WD,0,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013,1985
4,17,19,23,4,2.199219,4-Wheel or All-Wheel Drive,"(FFS,TRBO)",2700,Subaru,Legacy AWD Turbo,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1993
5,21,22,24,4,1.799805,Front-Wheel Drive,(FFS),1900,Subaru,Loyale,0,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013,1993
6,22,25,29,4,1.799805,Front-Wheel Drive,(FFS),1700,Subaru,Loyale,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1993
7,23,24,26,4,1.599609,Front-Wheel Drive,(FFS),1750,Toyota,Corolla,0,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013,1993
8,23,26,31,4,1.599609,Front-Wheel Drive,(FFS),1600,Toyota,Corolla,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1993
9,23,25,30,4,1.799805,Front-Wheel Drive,(FFS),1700,Toyota,Corolla,0,Automatic 4-spd,Tue Jan 01 00:00:00 EST 2013,1993


In [ ]:
#None of these operations are "in place" or destructive, 
# so to see their effects on memory usage, 
# you must rerun the commands assign and astype.
(autos
 #[cols_subset]
 .loc[:,cols_subset]
  .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'))
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16'})
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

17590123

17.6M of 19.6M is a memory savings of between 9 and 10 percent!

##Objects


In [ ]:
(autos
 [cols_subset]
 .select_dtypes('object')
 ).head(10)

,drive,eng_dscr,make,model,trany,createdOn
0,Rear-Wheel Drive,(FFS),Alfa Romeo,Spider Veloce 2000,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
1,Rear-Wheel Drive,(GUZZLER),Ferrari,Testarossa,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
2,Front-Wheel Drive,(FFS),Dodge,Charger,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
3,Rear-Wheel Drive,NaN,Dodge,B150/B250 Wagon 2WD,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013
4,4-Wheel or All-Wheel Drive,"(FFS,TRBO)",Subaru,Legacy AWD Turbo,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
5,Front-Wheel Drive,(FFS),Subaru,Loyale,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013
6,Front-Wheel Drive,(FFS),Subaru,Loyale,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
7,Front-Wheel Drive,(FFS),Toyota,Corolla,Automatic 3-spd,Tue Jan 01 00:00:00 EST 2013
8,Front-Wheel Drive,(FFS),Toyota,Corolla,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013
9,Front-Wheel Drive,(FFS),Toyota,Corolla,Automatic 4-spd,Tue Jan 01 00:00:00 EST 2013


* Are any columns categorical?
   - drive, make, model, trany
   - eng_dscr?
* Are any data entry formats problematic?
   - eng_dscr has multiple conventions
   - createdOn is date (datetime).

In [ ]:
# Inspect Drive 
(autos.drive.value_counts(dropna=False))
# 8 unique values and NaN.

Front-Wheel Drive             14236
Rear-Wheel Drive              13831
4-Wheel or All-Wheel Drive     6648
All-Wheel Drive                3015
4-Wheel Drive                  1460
NaN                            1189
2-Wheel Drive                   507
Part-time 4-Wheel Drive         258
Name: drive, dtype: int64

In [ ]:
#What cars do not have a drive? Else, what values are missing?
autos_abbr[autos_abbr.drive.isna()].head(10)

,city08,comb08,highway08,cylinders,displ,drive,eng_dscr,fuelCost08,make,model,range,trany,createdOn,year
7138,81,85,91,NaN,NaN,NaN,NaN,800,Nissan,Altra EV,90,NaN,Tue Jan 01 00:00:00 EST 2013,2000
8144,74,65,58,NaN,NaN,NaN,NaN,1000,Ford,Th!nk,29,NaN,Tue Jan 01 00:00:00 EST 2013,2001
8147,84,75,66,NaN,NaN,NaN,NaN,900,Nissan,Hyper-Mini,33,NaN,Tue Jan 01 00:00:00 EST 2013,2001
18217,18,21,25,4.0,2.0,NaN,(FFS),2000,Alfa Romeo,Spider Veloce 2000,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1984
18218,20,22,26,4.0,1.5,NaN,(FFS),1900,Bertone,X1/9,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1984
18219,13,15,20,8.0,5.7,NaN,(350 V8) (FFS),2800,Chevrolet,Corvette,0,Automatic 4-spd,Tue Jan 01 00:00:00 EST 2013,1984
18220,13,15,20,8.0,5.7,NaN,(350 V8) (FFS),2800,Chevrolet,Corvette,0,Manual 4-spd,Tue Jan 01 00:00:00 EST 2013,1984
18221,15,17,20,6.0,3.0,NaN,"(FFS,TRBO)",2500,Nissan,300ZX,0,Automatic 4-spd,Tue Jan 01 00:00:00 EST 2013,1984
18222,16,18,20,6.0,3.0,NaN,(FFS),2350,Nissan,300ZX,0,Automatic 4-spd,Tue Jan 01 00:00:00 EST 2013,1984
18223,16,18,22,6.0,3.0,NaN,"(FFS,TRBO)",2350,Nissan,300ZX,0,Manual 5-spd,Tue Jan 01 00:00:00 EST 2013,1984


In [ ]:
#Were there years when this data (drive) was not available?
#That is, are all 1984 ... missing drives?
(autos[cols_subset]
  .groupby('year')
  .drive
  .nunique()
 ).head(5)
#ANSWER: No, it occurs over decades. It's not the case that all 1984 cars are missing drives.

year
1984    3
1985    4
1986    4
1987    3
1988    3
Name: drive, dtype: int64

In [ ]:
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'),
         drive=autos.drive.fillna('Other').astype('category'))
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

12093275

In [ ]:
1 - (12.1/19.6)

0.38265306122448983

12.1M of 19.6M is a memory savings of 38 percent!

The improved dataset will enable and accelerate Machine Learning, reduce storage, reduce energy consumption, and costs of storage and processing. In streaming services, it will also improve efficiency.

In [ ]:
autos.trany.value_counts(dropna=False)
#This column hosts two pieces of data: whether automatic or manual and speed.
#Convert trany to two columns, automatic (T/F) and speeds, then drop trany.
#11 NaNs may also be coded with a default value (4) and the column type converted to category.

Automatic 4-spd                     11047
Manual 5-spd                         8361
Automatic 3-spd                      3151
Automatic (S6)                       3106
Manual 6-spd                         2757
Automatic 5-spd                      2203
Automatic (S8)                       1665
Automatic 6-spd                      1619
Manual 4-spd                         1483
Automatic (S5)                        833
Automatic (variable gear ratios)      826
Automatic 7-spd                       724
Automatic 8-spd                       433
Automatic (AM-S7)                     424
Automatic (S7)                        327
Automatic 9-spd                       293
Automatic (AM7)                       245
Automatic (S4)                        233
Automatic (AV-S6)                     208
Automatic (A1)                        201
Automatic (AM6)                       151
Automatic (AV-S7)                     139
Automatic (S10)                       124
Automatic (AM-S6)                 

In [ ]:
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'),
         drive=autos.drive.fillna('Other').astype('category'),
         #automatic will be boolean evaluation with str.contains.
         automatic=autos.trany.str.contains('Auto'),
         #speed will be string digit extracted with regex and converted to integer
         speed = autos.trany.str.extract(r'(\d)+').fillna('4').astype('int8'))
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
 .drop(columns=['trany'])
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

10631047

In [ ]:
1 - (10.6/19.6)

0.4591836734693878

10.6M of 19.6M is a memory savings of 46 percent!

## Dates

In [ ]:
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'),
         drive=autos.drive.fillna('Other').astype('category'),
         #automatic will be boolean evaluation with str.contains.
         automatic=autos.trany.str.contains('Auto'),
         #speed will be string digit extracted with regex and converted to integer
         speed = autos.trany.str.extract(r'(\d)+').fillna('4').astype('int8'),
         #the date column needs to be converted to datetime format. Use .replace in response to warnings.
         createdOn = pd.to_datetime(autos.createdOn.replace(
             {' EDT':'-04:00',' EST':'-05:00'},regex=True))
         ) #END assign
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
 .drop(columns=['trany'])
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

9437871

In [ ]:
1 - (9.4/19.6) 

0.5204081632653061

9.4M of 19.6M is a memory savings of 52 percent!

##Text Transformation with Regex

In [ ]:
#Inspect Engine Description
#Are values freeform, multifarious, and/or statistically unanalysable?
#Are there meaningful descriptions that should remain.
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'),
         drive=autos.drive.fillna('Other').astype('category'),
         #automatic will be boolean evaluation with str.contains.
         automatic=autos.trany.str.contains('Auto'),
         #speed will be string digit extracted with regex and converted to integer
         speed = autos.trany.str.extract(r'(\d)+').fillna('4').astype('int8'),
         #the date column needs to be converted to datetime format. Use .replace in response to warnings.
         createdOn = pd.to_datetime(autos.createdOn.replace(
             {' EDT':'-04:00',' EST':'-05:00'},regex=True))
         ) #END assign
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
 .drop(columns=['trany'])
 .eng_dscr
 .value_counts(dropna=False)
 )
#ANSWERS
#The column is free form text with multifarious entries.
#FFS seems to be the most meaningful term.
#Add an FFS column with boolean values and drop eng_dscr. 

NaN                                 16153
(FFS)                                8827
SIDI                                 5526
(FFS) CA model                        926
(FFS)      (MPFI)                     734
FFV                                   701
(FFS,TRBO)                            666
(350 V8) (FFS)                        411
(GUZZLER)  (FFS)                      366
SOHC                                  354
                                    ...  
B234L/R4 (FFS,TRBO)                     1
GUZZLER V8 FFS,TURBO                    1
4.6M FFS MPFI                           1
CNG FFS                                 1
POLICE FFS MPFI                         1
B308E5 FFS,TURBO                        1
5.4E-R FFS MPFI                         1
V-6 FFS                                 1
(GUZZLER)  (FFS)      (S-CHARGE)        1
R-ENG (FFS,TRBO)                        1
Name: eng_dscr, Length: 558, dtype: int64

In [ ]:
(autos
 [cols_subset]
 .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
         displ=autos.displ.fillna(0).astype('float16'),
         drive=autos.drive.fillna('Other').astype('category'),
         #automatic will be boolean evaluation with str.contains.
         automatic=autos.trany.str.contains('Auto'),
         #speed will be string digit extracted with regex and converted to integer
         speed = autos.trany.str.extract(r'(\d)+').fillna('4').astype('int8'),
         #the date column needs to be converted to datetime format. Use .replace in response to warnings.
         createdOn = pd.to_datetime(autos.createdOn.replace(
             {' EDT':'-04:00',' EST':'-05:00'},regex=True)),
         #ffs will be boolean with str.contains('FFS')
         ffs = autos.eng_dscr.str.contains('FFS')
         ) #END assign
 .astype({'highway08':'int8','city08':'int16','comb08':'int16',
          'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
 .drop(columns=['trany','eng_dscr'])
 .memory_usage(deep=True)
 .sum() #was 19,647,323
 )

8676214

In [ ]:
1 - (8.7/19.6)

0.556122448979592

8.7M of 19.6M is a memory savings of 56 percent!

## Functional Data Transformation

In [ ]:
#Write the pipeline above as a function. Take the original dataset
#as input and return a revised copy. 
def prep_autos(autos):
  #Define the columns of interest.
  cols_subset = ['city08','comb08','highway08','cylinders',
                'displ','drive','eng_dscr','fuelCost08','make',
                'model','range','trany','createdOn','year']
  
  #Return the transformed dataset.
  return (autos
          [cols_subset]
          .assign(cylinders = autos.cylinders.fillna(0).astype('int8'),
                  displ = autos.displ.fillna(0).astype('float16'),
                  drive = autos.drive.fillna('Other').astype('category'),
                  #automatic will be boolean evaluation with str.contains.
                  automatic = autos.trany.str.contains('Auto'),
                  #speed will be string digit extracted with regex and converted to integer
                  speed = autos.trany.str.extract(r'(\d)+').fillna('4').astype('int8'),
                  #the date column needs to be converted to datetime format. Use .replace in response to warnings.
                  createdOn = pd.to_datetime(autos.createdOn.replace(
                      {' EDT':'-04:00',' EST':'-05:00'},regex=True)),
                  #ffs will be boolean with str.contains('FFS')
                  ffs = autos.eng_dscr.str.contains('FFS')
                  ) #END assign
          .astype({'highway08':'int8','city08':'int16','comb08':'int16',
                    'fuelCost08':'int16','range':'int16','year':'int16', 'make': 'category'})
          .drop(columns=['trany','eng_dscr'])
  )

In [ ]:
autos_prepped = prep_autos(autos)
autos_prepped.head(10)

,city08,comb08,highway08,cylinders,displ,drive,fuelCost08,make,model,range,createdOn,year,automatic,speed,ffs
0,19,21,25,4,2.000000,Rear-Wheel Drive,2000,Alfa Romeo,Spider Veloce 2000,0,2013-01-01 00:00:00-05:00,1985,False,5,True
1,9,11,14,12,4.898438,Rear-Wheel Drive,3850,Ferrari,Testarossa,0,2013-01-01 00:00:00-05:00,1985,False,5,False
2,23,27,33,4,2.199219,Front-Wheel Drive,1550,Dodge,Charger,0,2013-01-01 00:00:00-05:00,1985,False,5,True
3,10,11,12,8,5.199219,Rear-Wheel Drive,3850,Dodge,B150/B250 Wagon 2WD,0,2013-01-01 00:00:00-05:00,1985,True,3,NaN
4,17,19,23,4,2.199219,4-Wheel or All-Wheel Drive,2700,Subaru,Legacy AWD Turbo,0,2013-01-01 00:00:00-05:00,1993,False,5,True
5,21,22,24,4,1.799805,Front-Wheel Drive,1900,Subaru,Loyale,0,2013-01-01 00:00:00-05:00,1993,True,3,True
6,22,25,29,4,1.799805,Front-Wheel Drive,1700,Subaru,Loyale,0,2013-01-01 00:00:00-05:00,1993,False,5,True
7,23,24,26,4,1.599609,Front-Wheel Drive,1750,Toyota,Corolla,0,2013-01-01 00:00:00-05:00,1993,True,3,True
8,23,26,31,4,1.599609,Front-Wheel Drive,1600,Toyota,Corolla,0,2013-01-01 00:00:00-05:00,1993,False,5,True
9,23,25,30,4,1.799805,Front-Wheel Drive,1700,Toyota,Corolla,0,2013-01-01 00:00:00-05:00,1993,True,4,True
